<a href="https://colab.research.google.com/github/Rehan6541/AI/blob/main/sentiment_analysis_transformers_detailed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.metrics import confusion_matrix,roc_curve
import seaborn as sns
import re
import string
from tensorflow.keras.layers import (Dense,Embedding,TextVectorization,Input)
from tensorflow.keras.models import Model
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
#load dataset
train_ds,val_ds,text_ds=tfds.load(name="imdb_reviews",split=["train","test[:60%]","test[50%:]"],as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.H6E3TM_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.H6E3TM_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.H6E3TM_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


tfds.load:loads the imdb movie data set from tensorfloe dataset

split divides the data into training,validation and  tests sets

as_supervised=true ensires the dataset returns (input,label) pairs

In [ ]:
#Text standardizaton
def standardization(input_data):
    lowercase=tf.strings.lower(input_data)
    no_tag=tf.strings.regex_replace(lowercase,"<[^>]+>"," ")
    return tf.strings.regex_replace(no_tag,"[%s]"%re.escape(string.punctuation)," ")

tf.strings.lower:converts text to lowercase

regex.replace:remoces html tags(<...>) and punctuation

re.escape(string.punctuation) matches all punctuation characters

In [ ]:
#Text vectorization parameters
VOCAB_SIZE=10000
SEQUENCE_LENGTH=250
vectorize_layer=TextVectorization(
    standardize=standardization,
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=SEQUENCE_LENGTH
)

TextVectorization converts text into numerical sequence for u=input to the model

standardize=standardization:uses the custom standardization function.

max_tokens=10000:limits the voocabulary size

output_sequence_length=250:pads/truncates sequence to 250 tokens

In [ ]:
#Prepare vectorization
training_data=train_ds.map(lambda x,y: x)
vectorize_layer.adapt(training_data)

def vectorizer(review,label):
    return vectorize_layer(review),label

train_ds.map(lambda x,y:x):extracts only the text(x) from the dataset

vectorize_layer.adapt(training_data) learns the vocabulaty from the training data




In [ ]:
# Prepare training data for vectorization
training_data = train_ds.map(lambda x, y: x).unbatch().cache()

# Use a subset of the data if adaptation is slow
training_data_for_adapt = training_data.take(1000)

# Adapt the vectorize_layer
vectorize_layer.adapt(training_data_for_adapt)

# Define the vectorizer function
def vectorizer(review, label):
    return vectorize_layer(review), label


ValueError: Cannot unbatch an input with scalar components.

In [ ]:
#Vectorize datasets
train_dataset=train_ds.map(vectorizer).batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset=val_ds.map(vectorizer).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset=test_ds.map(vectorizer).batch(32)

NameError: name 'test_ds' is not defined

vectorizer applies the text vectorizaton and retrns(vectorized text,label)

.batch(32) groups data into batches of size 32 for efficient processing

prefetch(tf.data.AUTOTUNE optimizes data loading for faster training